In [ ]:
# Check documentation: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
#
# Video about this notebook: 
# https://youtu.be/l9S6sWCykDo

In [24]:
import pandas as pd

In [25]:
dfTestCSV = pd.DataFrame(pd.read_csv('./dfanalyticstestdata1.csv', header=2, sep="\t", index_col=0))

In [26]:
dfTestCSV['ID_GA'] = dfTestCSV['ga:pagePath'].str.extract('(?:.*id)([0-9]+)(?:.html)')

In [27]:
dfTestCSV = pd.pivot_table(dfTestCSV,index='ID_GA', values='ga:pageviews', aggfunc=['count', 'sum'])
dfTestCSV.reset_index(inplace=True)
dfTestCSV.columns = dfTestCSV.columns.droplevel(1)
dfTestCSV = dfTestCSV.rename(columns={"count": "id count", "sum": "pageviews per id"})
dfTestCSV.head(10)

,ID_GA,id count,pageviews per id
0,10061,2,8
1,1510256,1,1
2,151553,1,5
3,1742,1,1
4,19186,1,28
5,19969,1,2
6,2241980,1,1
7,22672,1,1
8,2483959,1,1
9,26545,1,1


In [30]:
dfCrawled = pd.DataFrame(pd.read_excel('./custom_search_all-test.xlsx', header=0, sheet_name='1 - All'))
dfCrawled = dfCrawled.rename(columns={"Contains: \'\"article-text-box\"\'": "article-text-box", 
                                         "Contains: \'\"video-player\"\'":"video-player", 
                                         "Contains: \'\"article-body\'":"article-body",
                                         "Contains: \'\(sda\)\'":"sda",
                                         "Contains: \'\"scribble\"\'":"scribble",
                                         "Contains: \'basic-image--free-desktop\'":"has-image", 
                                         "Contains: \'<iframe\'":"iframe", 
                                         "Contains: \'layout-item--flex-desktop-4'":"normal-article-layout", 
                                         "Contains: \'\(sda\/\'":"sda2", 
                                     })
dfCrawled['ID_Crawl'] = dfCrawled['Address'].str.extract('(?:.*id)([0-9]+)(?:.html)', expand=False)
dfCrawled = dfCrawled.drop("Status Code",1)
dfCrawled = dfCrawled.drop("Status",1)
dfCrawled = dfCrawled.drop("Content",1)
dfCrawled.head(10)

,Address,article-text-box,video-player,article-body,sda,scribble,has-image,iframe,normal-article-layout,sda2,ID_Crawl
0,https://www.blick.ch/people-tv/international/r...,0,0,0,0,0,0,0,0,0,4270807
1,https://www.blick.ch/sport/eishockey/overtime-...,0,0,0,0,0,0,0,0,0,106244
2,https://www.blick.ch/news/ausland/neben-berueh...,0,0,0,0,0,0,0,0,0,3896239
3,https://www.blick.ch/news/grausiger-fund-in-an...,0,0,0,0,0,0,0,0,0,2024473
4,https://www.blick.ch/news/mahamaya-bern-die-we...,0,0,0,0,0,0,0,0,0,110094
5,https://www.blick.ch/people-tv/musik/liechtens...,0,0,0,0,0,0,0,0,0,2131248
6,https://www.blick.ch/people-tv/tv/bachelorette...,0,0,0,0,0,0,0,0,0,3790667
7,https://www.blick.ch/news/das-ist-die-swiss-bo...,0,0,0,0,0,0,0,0,0,2238005
8,https://www.blick.ch/news/schweiz/vergangenhei...,0,0,0,0,0,0,0,0,0,5550392
9,https://www.blick.ch/news/wirtschaft/das-sind-...,0,0,0,0,0,0,0,0,0,4955860


In [29]:
dfMerged = pd.merge(dfTestCSV, dfCrawled, left_on='ID_GA', right_on='ID_Crawl', how='outer')
dfMerged = dfMerged[dfMerged['ID_Crawl'].notnull()]
dfMerged = dfMerged[dfMerged['ID_GA'].notnull()]
dfMerged = dfMerged.drop("ID_Crawl",1)
dfMerged.head(10)

,ID_GA,id count,pageviews per id,Address,article-text-box,video-player,article-body,sda,scribble,has-image,iframe,normal-article-layout,sda2
34,31000,1.0,3.0,https://www.blick.ch/people-tv/international/e...,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,2.0
36,31002,1.0,1.0,https://www.blick.ch/news/ausland/afghanistan-...,0.0,0.0,1.0,3.0,2.0,0.0,1.0,1.0,0.0
37,31004,1.0,5.0,https://www.blick.ch/news/zwei-schweizer-in-ve...,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0
51,31275,2.0,6.0,https://www.blick.ch/news/schweiz/zuerich/knas...,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0
52,31277,1.0,12.0,https://www.blick.ch/news/ausland/steinmeier-p...,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0
65,31388,1.0,3.0,https://www.blick.ch/news/schweiz/zentralschwe...,0.0,0.0,5.0,0.0,2.0,0.0,1.0,1.0,0.0
68,31425,1.0,1.0,https://www.blick.ch/people-tv/international/h...,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0
69,31426,1.0,1.0,https://www.blick.ch/sport/eishockey/nla/serve...,1.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0
70,31427,1.0,1.0,https://www.blick.ch/digital/games/news/colin-...,0.0,0.0,3.0,0.0,2.0,5.0,1.0,1.0,0.0
71,31428,1.0,1.0,https://www.blick.ch/digital/games/larry-magna...,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0
